In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, create_sql_agent, get_all_tool_names
from langchain.chains import SQLDatabaseChain
from langchain import OpenAI, SerpAPIWrapper, LLMChain, GoogleSearchAPIWrapper
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain, LLMMathChain
from langchain.utilities import PythonREPL

from chains.co2_sql import get_en_co2_sql_chain
from chains.weight_est import get_en_weight_est

In [ ]:
search_chain = GoogleSearchAPIWrapper(k=10, search_engine="google")
math_chain = LLMMathChain(llm=ChatOpenAI(temperature=0)) # type: ignore
sql_chain = get_en_co2_sql_chain()
weight_est_chain = get_en_weight_est()


tools = [
    Tool(
        name="Weight estimator",
        func=weight_est_chain.run,
        description="Useful for estimating the weight of each item by passing all ingredients at once as input.",
    ),
    Tool(
        name="CO2 emission estimator",
        func=sql_chain.run,
        description="Useful for finding out the CO2 emission of each item by passing all ingredients at once as input.",
    ),
    Tool(
        name="Google search",
        func=search_chain.run,
        description="Useful for finding out the kg CO2e / kg if the ingredient is not in the database. Should only use if the ingredient weights more than 100 g.",
    ),
    Tool(
        name="Math calculator",
        func=math_chain.run,
        description="""Useful for calculating the CO2 emission of each ingredients by kg * kg CO2e / kg. You can for example ask: What is 0.1 * 1.26 + 0.2 * 0.5?"""
    )
]

In [ ]:
prefix = """Act as a CO2 emission calculator that understands danish and english. 

Given a list of ingredients to a recipe, your job is to estimate the CO2 emission for each ingredient in kg CO2e and the recipe in total. 
It is important that you include the intermediate calculations for each ingredients and the total CO2 emission of the recipe 
in the final answer.
 
You have access to the following tools:"""

In [ ]:
prefix = """Act as a CO2 emission calculator that understands danish and english. 

Given a list of ingredients to a recipe, your job is to estimate the CO2 emission for each ingredient in kg CO2e and the recipe in total. 
 
You have access to the following tools:"""

suffix = """

You can use all tools as many times as you want.

You solve this task by: 
1. Estimating the weights of the ingredients in the recipe using the weight estimation tool. This steps also defines what ingredients are neglible.
2. Finding the CO2 emission of each ingredient in kg CO2e per kg using the CO2 emission estimator tool. 
   Remember querying all ingredients. 
3. If necessary, use Google Search to get the kg CO2e per kg of a ingredient if the ingredient is not found in step 2 
   and not is neglible. You should search for one ingredient at a time.
4. Calculate the CO2 emission in kg per ingredient and sum them to get the total CO2 emission of the recipe using the math calculator tool.
5. Provide the answer where you show the CO2 calculation for each ingredient and the total CO2 emission of the recipe. 

Small amounts of ingredients like salt, sugar etc. that weight less than 0.01 kg can be ignored.

You must provide the Final Answer in the following format: 

Total CO2 emission: Z Total kg CO2e
Calculation method per ingredient: X kg CO2e / kg * Y kg = Z kg CO2e
ingredient 1: X1 * Y1 = Z1 OR neglible if ingredient is neglible in step 1
ingredient 2: X2 * Y2 = Z2 OR neglible if ingredient is neglible in step 1
etc. 

Here is an example: 
Total CO2 emission: 0.267 Total kg CO2e
Calculation method per ingredient: X kg CO2e / kg * Y kg = Z kg CO2e
Red lentils: 1.78 * 0.15 = 0.267 
Salt: neglible

In the Question section, the ingredients are provided. 

Begin! 

Question: {input}
{agent_scratchpad}"""

In [ ]:
prefix = """Act as a CO2 emission calculator that understands danish and english. 

Given a list of ingredients to a recipe, your job is to estimate the CO2 emission for each ingredient in kg CO2e and the recipe in total. 
 
You have access to the following tools:"""

suffix = """

You can use all tools as many times as you want.

You solve this task by: 
1. Estimating the weights of the ingredients in the recipe using the weight estimation tool. This steps also defines what ingredients are neglible.
2. Finding the CO2 emission of each ingredient in kg CO2e per kg using the CO2 emission estimator tool. 
   Remember querying all ingredients. 
3. If necessary, use Google Search to get the kg CO2e per kg of a ingredient if the ingredient is not found in step 2 
   and not is neglible. You should search for one ingredient at a time.
4. Calculate the CO2 emission in kg per ingredient and sum them to get the total CO2 emission of the recipe using the math calculator tool.
5. Provide the answer where you show the CO2 calculation for each ingredient and the total CO2 emission of the recipe. 

Small amounts of ingredients like salt, sugar etc. that weight less than 0.01 kg can be ignored.

You must provide the Final Answer in the following format: 

Total CO2 emission: Z Total kg CO2e
Calculation method per ingredient: X kg CO2e / kg * Y kg = Z kg CO2e
ingredient 1: X1 * Y1 = Z1 OR neglible if ingredient is neglible in step 1
ingredient 2: X2 * Y2 = Z2 OR neglible if ingredient is neglible in step 1
etc. 

In the Question section, the ingredients are provided. 

Begin! 

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [ ]:
from langchain.chat_models import ChatOpenAI

llm_chain = LLMChain(
    # llm=OpenAI(temperature=0),
    llm=ChatOpenAI(temperature=0),
    prompt=prompt,
)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    answer = agent_executor.run(
        """
        150 g red lentils
        1 large onion, 1 pepper
        2 cubes of vegetable bouillon
        3 cloves garlic, pressed
        1 can of chopped tomatoes,
        1 tin of tomato concentrate (140 g),
        1 can of coconut milk (400 ml),
        1 tbsp. lemon juice,
        1 tsp. sugar,
        ½ teaspoon chili powder,
        ½ teaspoon ground cumin,
        Possibly. 1/4 tsp. smoked paprika,
        A large handful of fresh basil, chopped
        (or 2 tsp dried basil)
        """
    )
    
    print("Total Tokens:", cb.total_tokens)
    print("Total prompt tokens:", cb.prompt_tokens)
    
    

In [ ]:
print(answer)

## Test in conjunction with URL ingredient extractor

In [ ]:
from chains.recipe_extractor import get_recipe_extractor_chain
from utils import get_url_text

recipe_extractor_chain = get_recipe_extractor_chain(verbose=True)

In [ ]:
# url = "https://www.valdemarsro.dk/blinis-med-stenbiderrogn/"
url = "https://www.foodfanatic.dk/tacos-med-lynchili-og-salsa"

text = get_url_text(url)
ingredients = recipe_extractor_chain.run(text)

In [ ]:
agent_executor.run(ingredients)